## **`Web Scrapper`**
#### Zillow - Reading a list of addresses from a google sheet and scrapping additional details on each property from zillow.com

In [1]:
pip install unicodecsv

  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-cp36-none-any.whl size=10769 sha256=18407fc26038a3c174894d413d2307009fe3e5e6d37e23abd8584a7b3386ea77
  Stored in directory: /root/.cache/pip/wheels/a6/09/e9/e800279c98a0a8c94543f3de6c8a562f60e51363ed26e71283
Successfully built unicodecsv


In [5]:
"""**Import all the libraries**"""

from lxml import html
import requests
import unicodecsv as csv
import argparse
import json
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from collections import OrderedDict
import pandas as pd
from io import BytesIO
import requests

In [6]:
r = requests.get('https://docs.google.com/spreadsheets/d/e/2PACX-1vQgYTKQBRlL6cYPL3PuaIDH03Hdbohycj8Y_thlLwkEV0ZFMnQBWVJYY9_Q8HEHfi3jxWXPWQKhmWB8/pub?output=tsv')
df = pd.read_csv(BytesIO(r.content), sep='\t')
df.columns

fulladdress = df["Full Address"] +" "+ df["Zip"].map(str)
mlsDF = df["MLS #"]

print(fulladdress)
print(mlsDF)
fulladdress.shape

fulladdress = fulladdress.str.replace('  ',' ')
fulladdress = fulladdress.str.replace(' ','-')
print(fulladdress.head)

0       4095 Bonita Road  132 91902
1           88 Fourth Ave  10 91910
2      2167 Palo Alto Dr  133 91914
3        2134 Cantata Dr.  36 91914
4        280 Salinas Dr.  171 91914
                   ...             
242         7019 DEERHURST CT 92139
243          2233 Manzana Way 92139
244    24374 Larchmont Ct  69 92653
245       25185 Via Terracina 92677
246      3575 Cameo Drive  43 92056
Length: 247, dtype: object
0      180045508
1      180024056
2      180035693
3      180042493
4      180038189
         ...    
242    180044441
243    180044784
244    180040324
245    180042450
246    180035417
Name: MLS #, Length: 247, dtype: int64
<bound method NDFrame.head of 0       4095-Bonita-Road-132-91902
1           88-Fourth-Ave-10-91910
2      2167-Palo-Alto-Dr-133-91914
3        2134-Cantata-Dr.-36-91914
4        280-Salinas-Dr.-171-91914
                  ...             
242        7019-DEERHURST-CT-92139
243         2233-Manzana-Way-92139
244    24374-Larchmont-Ct-69-92653
245   

In [9]:
def clean(text):
    if text:
        return ' '.join(' '.join(text).split())
    return None

def get_headers():
    # Creating headers.
    headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
               'accept-encoding': 'gzip, deflate, sdch, br',
               'accept-language': 'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
               'cache-control': 'max-age=0',
               'upgrade-insecure-requests': '1',
               'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    return headers

def create_url(address):
    url = "https://www.zillow.com/homes/"+address+"_rb/"
    print(url)
    return url

def save_to_file(response):
    # saving response to `response.html`

    with open("response.html", 'w') as fp:
        fp.write(response.text)

def write_data_to_csv(data):
    # saving scraped data to csv.

    with open("properties-%s.csv" % ('final'), 'wb') as csvfile:
        fieldnames = ['MLS #', 'HOA', 'Zestimate', 'Sold', 'Sold On', 'Rent Estimate', 
                      'Mortgage', 'Walk Score', 'Transit Score', 'Neighborhood', 
                      'Neighborhood Change In Price', 'Neighborhood Predicted Change','Neighborhood Median Price']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

def get_response(url):
    # Getting response from zillow.com.

    for i in range(5):
        response = requests.get(url, headers=get_headers())
        print("status code received:", response.status_code)
        if response.status_code != 200:
            # saving response to file for debugging purpose.
            save_to_file(response)
            continue
        else:
            save_to_file(response)
            return response
    return None

def parse(mls, address):
    url = create_url(address)
    response = get_response(url)

    if not response:
        print("Failed to fetch the page, please check `response.html` to see the response received from zillow.com.")
        return None

    # These two new lines are added
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    parser = html.fromstring(webpage)

    #overview = parser.xpath("//*[@id='ds-data-view']/ul/li[4]/div[1]/div/div[1]/div[1]/div//text()")
    #print(overview)
    hoa = parser.xpath("//*[@id='ds-data-view']/ul/li[4]/div[2]/div/div[1]/ul/li[6]/span[2]//text()")
    #print('hoa ', hoa)
    zestimate = parser.xpath("//*[@id='ds-container']/div[4]/div[2]/div/p/span[3]/span[2]/span[2]//text()")
    #print('Zestimate ', zestimate)
    sold = parser.xpath("//*[@id='ds-container']/div[4]/div[2]/div/p/span[1]/span[2]/text()[2]")
    #print('Sold Price ', sold)
    soldOn = parser.xpath("//*[@id='ds-container']/div[4]/div[2]/div/p/span[2]/text()[2]")
    #print('Sold On', soldOn)
    rentEstimate = parser.xpath("//*[@id='ds-data-view']/ul/li[3]/div[2]/div[1]/div[1]/div[3]/p[1]/span//text()")
    #print('Rent Estimate', rentEstimate)
    mortgage = parser.xpath("//*[@id='ds-data-view']/ul/li[3]/div[2]/div[1]/div[2]/div[3]/p[1]/span//text()")
    #print('Mortgage ', mortgage)
    walkScore = parser.xpath("//*[@class='ds-data-view']/ul/li[5]/div[1]/div[1]/div/ul/li[2]/a/span//text()")
    #print('Walk Score 11 ', walkScore)
    transitScore = parser.xpath("//*[@id='ds-data-view']/ul/li[5]/div[1]/div[1]/div/ul/li[2]/span[2]/text()[2]")
    #print('Transit Score ', transitScore)
    changeInPastYear1 = parser.xpath("//*[@id='ds-data-view']/ul/li[5]/div[1]/div[1]/ul/li[1]/p/span/text()[1]")
    changeInPastYear2 = parser.xpath("//*[@id='ds-data-view']/ul/li[5]/div[1]/div[1]/ul/li[1]/p/span/text()[2]") 
    changeInPastYear = changeInPastYear1 + changeInPastYear2
    #print("changeInPastYear ", changeInPastYear)
    expectedChangeInValue1 = parser.xpath("//*[@id='ds-data-view']/ul/li[5]/div[1]/div[1]/ul/li[2]/p/span/text()[1]")
    expectedChangeInValue2 = parser.xpath("//*[@id='ds-data-view']/ul/li[5]/div[1]/div[1]/ul/li[2]/p/span/text()[3]")
    expectedChangeInValue = expectedChangeInValue1 + expectedChangeInValue2
    #print("expectedChangeInValue ", expectedChangeInValue)
    neighborhood = parser.xpath("//*[@id='ds-container']/div[4]/div[2]/div/div[1]/header/h1/span[2]/text()[2]")
    #print("neighborhood ", neighborhood)
    neighborhoodMedianPrice = parser.xpath("//*[@id='ds-data-view']/ul/li[5]/div[1]/div[1]/ul/li[4]/p/text()[4]")
    #print("neighborhoodMedianPrice ", neighborhoodMedianPrice)

    # soup = BeautifulSoup(r.content, 'html.parser')
    # property_data = OrderedDict()
    # title_list = soup.findAll('span', {'class': 'Text-aiai24-0'})
    # print(title_list)
    # property_data['walkScore'] = soup.find('span', attrs={'class': 'ds-container'})
    # print("property_data['walkScore'] ", property_data['walkScore']) 

    #properties_list = []
    properties = {
                  'MLS #' : mls,
                  'HOA' : clean(hoa),
                  'Zestimate' : clean(zestimate),
                  'Sold' : clean(sold),
                  'Sold On': clean(soldOn),
                  'Rent Estimate' : clean(rentEstimate),
                  'Mortgage': clean(mortgage),
                  'Walk Score': clean(walkScore),
                  'Transit Score' : clean(transitScore),
                  'Neighborhood' : clean(neighborhood),
                  'Neighborhood Change In Price' : clean(changeInPastYear),
                  'Neighborhood Predicted Change' : clean(expectedChangeInValue),
                  'Neighborhood Median Price' : clean(neighborhoodMedianPrice)
    }
    return properties

In [10]:
properties_list = []

for i in fulladdress.index:
   print(mlsDF[i], fulladdress[i])
   scraped_data = parse(mlsDF[i], fulladdress[i])
   properties_list.append(scraped_data)

if properties_list:
  print ("Writing data to output file")
write_data_to_csv(properties_list)

180045508 4095-Bonita-Road-132-91902
https://www.zillow.com/homes/4095-Bonita-Road-132-91902_rb/
status code received: 200
180024056 88-Fourth-Ave-10-91910
https://www.zillow.com/homes/88-Fourth-Ave-10-91910_rb/
status code received: 200
180035693 2167-Palo-Alto-Dr-133-91914
https://www.zillow.com/homes/2167-Palo-Alto-Dr-133-91914_rb/
status code received: 200
180042493 2134-Cantata-Dr.-36-91914
https://www.zillow.com/homes/2134-Cantata-Dr.-36-91914_rb/
status code received: 200
180038189 280-Salinas-Dr.-171-91914
https://www.zillow.com/homes/280-Salinas-Dr.-171-91914_rb/
status code received: 200
180043165 2360-Corte-Pescado-88-91914
https://www.zillow.com/homes/2360-Corte-Pescado-88-91914_rb/
status code received: 200
180045558 2340-Corte-Flores-98-91914
https://www.zillow.com/homes/2340-Corte-Flores-98-91914_rb/
status code received: 200
180037792 2271-Alicante-Cove-91914
https://www.zillow.com/homes/2271-Alicante-Cove-91914_rb/
status code received: 200
180040474 381-Caminito-Barce